# SETI Signal Search - CNN - 26

## Specific

* in CNN-26 experiment with cutting off left and right side edges as a randomization technique using CNN-18 models)
  Work in Progress, this is only the base level without the cut-off left and right.
  Confirming the earlier results from CNN-20, which was using the models from CNN-18 (most successful 0.94
  submission up to now, without augmented duplication of positives, only randomization of existing training examples)
* in CNN-25 Plot ROC for oversampling 8 with noise 0.5 (CNN-22)
* in CNN-24 prepare data for Plot ROC for oversampling 8 with noise 0.5 (CNN-22)
* in CNN-23 for 1 fold, try noise at 1.0 (5 epochs, last epoch serious drop in AUC, probably too much noise)
* in CNN-22 we will try a simplistic technique duplicating the positive cases with some variations
* in CNN-21 we analysed the ROC of the initial training steps
* in CNN-20 we calculated inferences on epoch 00, 01, 02 and 05 for folds 0 and 1
* try to understand the ROC curve:
* CNN-19 3 epochs on the first 2 Folds training effv2 b1 from pretrained timm
* CNN-18 6th epoch on the all 5 Folds training effv2 b1 from pretrained timm
* comparing the curves for validation data in a Fold and training data in that Fold
  to see if we can see and/or understand overfitting.
  
## New ideas

* plot ROC for the oversampling case with 0.5
* plot predictions for full_width, partial_right, partial_left
* look at FN and FP
* plot 4 individual predictions vs. average prediction (before and after sigmoid) => and calculate AUC's for them


## Global

Try to predict the presence of "needles" with a CNN using PyTorch.

For transfer learning, look at TF EfficientNet and TF EfficientNet V2

In the list of Pytorch Image models https://paperswithcode.com/lib/timm/ and sorting them by TOP 1 Accuracy, the EfficientNet is the first model that goes under 10 Billion Flops. Also, there are variations from b0 to b8 that I presume will make it possible to trade-off compute cost vs. accuracy.

Very recently (14 May) the V2 was ported to this PyTorch repo. Maybe also testing tf_efficientnetv2_b0 up to tf_efficientnetv2_b3 ?

Inspired by https://www.kaggle.com/piantic/train-seti-pytorch-starter-chans-vs-spatial from https://www.kaggle.com/piantic

KFold and initial Convolutional filter inspired by Salman https://www.kaggle.com/micheomaano/mixup-training-5fold-spatial/execution

# Libraries

In [ ]:
import sys
sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')
import timm
print(timm.__version__)

import os
import datetime as dt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
from tqdm import tqdm

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast
from torch.optim import Adam

import cv2
import albumentations as A

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Config

In [ ]:
class CFG:
    debug = False

    unskew = False  # False will not trigger adding unskew training data
    training = False  # False will not trigger an actual training (e.g. do an evaluation)
    submission = False  # False will only calculate 1 Fold and not do submission

    input_model_cnn = "18"
    evaluation = True  # False will not trigger an evaluation of earlier loaded models
    
    input_inferences_cnn = "20"
    visualisation = True  # True will plot ROC curve etc.
    
    epochs = 5 # not really relevant if no training ...

    # focus on tf_efficientnetv2_b1 for now
    model_name = 'tf_efficientnetv2_b1'
    model_size = 192  # 3, 192, 192
    valid_model_size = 240  # 3, 240, 240

    batch_size = 64
    inference_batch_size = 64
    num_workers = 8

    criterion = nn.BCEWithLogitsLoss()

    seed = 45

    N_FOLDS = 4  # 4 instead of 5, since evaluation is more important to be more sure on the secret Kaggle test set
    p_horizontal_flip = 0.20  # Not too sure if this is really a good idea (the data seems asymmetric ...)
    random_gain_positives = 0.30  # a factor for reduction of gain on duplicated positives
    random_noise_positives = 0.50  # random noise adding on duplicated positives
    sample_width = 250  # of 256  => probably replace this with controlled side-cutting (CNN-26)

    lr = 8e-5  # maybe too much for final fine-tuning ?

### Hyper parameters

| run | folds |   lr  | horiz | width | rnd gain | rnd noise |
|----:|------:|------:|------:|------:|---------:|----------:|
| 22  |   4/4 |  8e-5 |  0.10 |   252 |     0.30 |      0.50 |
| 23  |   1/4 |  8e-5 |  0.20 |   250 |     0.30 |      1.00 |

# Import data

In [ ]:
import os

print("os.walk in part of /kaggle/input/")

def walk_kaggle_input(dir):
    for dirname, _, filenames in os.walk(f"/kaggle/input/{dir}/output"):
        for filename in sorted(filenames[0:100]):
            print(os.path.join(dirname, filename))

walk_kaggle_input(f"seti-signal-search-cnn-{CFG.input_model_cnn}")

In [ ]:
BASE_DIR = '/kaggle/input/seti-breakthrough-listen'

def get_file_path(image_id, category):
    return f"{BASE_DIR}/{category}/{image_id[0]}/{image_id}.npy"

def get_train_file_path(image_id):
    return get_file_path(image_id, "train")

def get_test_file_path(image_id):
    return get_file_path(image_id, "test")


In [ ]:
train_original = pd.read_csv(f"{BASE_DIR}/train_labels.csv")
ORIGINAL_SIZE = len(train_original)
train_original

In [ ]:
if CFG.unskew:
    train_positives = train_original[train_original['target'] == 1]
    train_unskewed = (
        train_original
        .append(train_positives)
        .append(train_positives)
        .append(train_positives)
        .append(train_positives)
        .append(train_positives)
        .append(train_positives)
        .append(train_positives)
        .append(train_positives)
        .append(train_positives)
        .reset_index(drop=True)
    )

    # Shuffle the train, to mix the positive duplicates inbetween originals 
    # drop False allows to know from where this row originates
    # index 0 => 50164: train_original
    # index > 50164: one of the extra rounds
    train = train_unskewed.sample(frac=1, random_state=42).reset_index(drop=False)
    train.rename(columns={'index': 'original_index'},inplace=True)
else:
    train = train_original.assign(original_index=train_original.index)
    
train

In [ ]:
# Add img_path and show class distribution for augmented train
train['img_path'] = train['id'].apply(get_train_file_path)

display(train.head(1))
print(train.head(1)['img_path'].values)

display(train['target'].value_counts())

In [ ]:
# Add img_path and show class distribution for train
test = pd.read_csv(f"{BASE_DIR}/sample_submission.csv")

test['img_path'] = test['id'].apply(get_test_file_path)

display(test.head(1))
print(test.head(1)['img_path'].values)

display(test['target'].value_counts())

In [ ]:
if CFG.debug:
    print('debug!')
    CFG.epochs = 1
    CFG.batch_size = 8
    CFG.inference_batch_size = 16
    CFG.num_workers = 4
    
    train = train.sample(n=193, random_state=CFG.seed).reset_index(drop=True)
    display(train['target'].value_counts())
    test = test.head(153)

# Modelling

Initial Exploratory Data Analysis was done in https://www.kaggle.com/peterv1/seti-signal-search-data-exploration/

Using the EfficientNet ports to Pytorch from Ross Wightman Ref. https://github.com/rwightman/pytorch-image-models

In [ ]:
# Make output dir
OUTPUT_DIR = './output/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Preprocessing

In [ ]:
ttransform = A.Compose([
    A.RandomCrop(height=1638, width=CFG.sample_width), # cut-off random 6 horizontally
    A.HorizontalFlip(p=CFG.p_horizontal_flip),
    A.Resize(CFG.model_size, CFG.model_size, cv2.INTER_NEAREST),
])
vtransform = A.Compose([
    A.Resize(CFG.valid_model_size, CFG.valid_model_size, cv2.INTER_NEAREST)
])

# Dataset

In [ ]:
import random
  
class ClassificationDataset:
    
    def __init__(self, img_paths, targets, original_indexes, tr): 
        self.img_paths = img_paths
        self.targets = targets
        self.original_indexes = original_indexes
        self.tr = tr

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, item):
        img_path = self.img_paths[item]
        image = np.load(img_path)
        image = np.vstack(image).astype(float)
        image = self.tr(image = image)["image"][np.newaxis, ]
        target = self.targets[item]

        # in training and for duplicated positive images, do a bit more transformations, make them harder
        if self.original_indexes is None:
            original_index = 0
        else:
            original_index = self.original_indexes[item]                        
            if (original_index >= ORIGINAL_SIZE):
                # reduce the gain
                zero_to_one = random.uniform(0.0, 1.0)
                image = image * (1 - CFG.random_gain_positives * zero_to_one)      
                noise = np.random.normal(0, CFG.random_noise_positives, image.shape)
                image = image + noise
            
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "target": torch.tensor(target, dtype=torch.float),
            "img_id": img_path.split('/')[-1].split('.')[0],
            "original_index": original_index,
        }

# Preview

In [ ]:
# Preview some training images via the ClassificationDataset

display(train)
X = train.img_path.values
y = train.target.values
i = train.original_index.values

sample_size = 2
train_index = 100 # some random image
train_images = X[train_index:train_index+sample_size]
train_targets = y[train_index:train_index+sample_size]
train_original_indexes = i[train_index:train_index+sample_size]

train_dataset = ClassificationDataset(
    img_paths=train_images,
    targets=train_targets,
    original_indexes=train_original_indexes,
    tr=ttransform
)

for i in range(sample_size):
    image_target = train_dataset[i]
    image, target, img_id, original_index = (
        image_target['image'],
        image_target['target'],
        image_target['img_id'],
        image_target['original_index']
    )
    # transpose back from torch format to imshow format
    plt.imshow(image.numpy().transpose((1, 2, 0))[:,:,0]) # only 1 axis
    plt.title(f"target: {target}, img_id: {img_id}, original_index: {original_index}, min: {image.min():.4f}, max: {image.max():.4f}")
    plt.show()
image.shape

In [ ]:
# Preview 2 test images via the ClassificationDataset
X = test.img_path.values
y = test.target.values

sample_size = 2
test_index = 27 # some random image
test_images = X[test_index:test_index+sample_size]
test_targets = y[test_index:test_index+sample_size]

test_dataset = ClassificationDataset(img_paths=test_images, targets=test_targets, original_indexes=None, tr=vtransform) # vtransform !

for i in range(sample_size):
    image_target = test_dataset[i]
    image, target = image_target['image'], image_target['target']
    # transpose back from torch format to imshow format
    plt.imshow(image.numpy().transpose((1, 2, 0))[:,:,0]) # only 1 axis
    plt.title(f'target: {target}')
    plt.show()
image.shape

# Model

In [ ]:
class timmv2(nn.Module):
    def __init__(self, model_name, pretrained):
        super().__init__()
        
        # Existing EfficientNet fixed at 3 channels
        self.enet = timm.create_model(model_name, pretrained=pretrained, in_chans=3)
        
        # Added a trainable 1 to 3 conv1 layer before
        # TODO try padding=1 
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=3, bias=True)
        
        # set the output classifier to 1 feature
        nb_ft = self.enet.classifier.in_features
        self.enet.classifier = nn.Linear(nb_ft, 1)

    @autocast()
    def forward(self, x):
        x = self.conv1(x)
        x = self.enet(x)
        
        return x

In [ ]:
def model_make(model_name):
    model = timmv2(model_name, True) # Start from pre-trained
    state_dict = {
        'weight':torch.tensor(
            [[[
                [ -0.03, 0.1,  -0.03],
                [ -0.03, 0.1,  -0.03],
                [ -0.03, 0.1,  -0.03],
            ]],[[
                [ -0.03, 0.1,  -0.03],
                [ -0.03, 0.1,  -0.03],
                [ -0.03, 0.1,  -0.03],
            ]],[[
                [ -0.03, 0.1,  -0.03],
                [ -0.03, 0.1,  -0.03],
                [ -0.03, 0.1,  -0.03],
            ]]], requires_grad=True    
        ),
        'bias':torch.tensor(
            [0.2, 0.2, 0.2], requires_grad=True
        )}
    model.conv1.load_state_dict(state_dict, strict=True)
    return model

In [ ]:
def model_make_custom(model_name, cnn_version, fold=0, epoch=-1):
    model = timmv2(model_name, False) # Start from SELF-trained
    
    prefix = f"/kaggle/input/seti-signal-search-cnn-{cnn_version}/output"
    if epoch >= 0:
        file_name = f"{prefix}/tf_efficientnetv2_b1_fold_{fold:02d}_epoch_{epoch:02d}_state.pth"
    else:
        file_name = f"{prefix}/tf_efficientnetv2_b1_fold_{fold:02d}_state.pth"
        
    # TODO: is map_location cuda OK when model is not yet loaded in GPU ?
    model.load_state_dict(torch.load(file_name, map_location=torch.device(device))['model'])    
    return model

In [ ]:
# Make a custom model and show the conv1 parameters

if CFG.evaluation:
    model = model_make_custom(CFG.model_name, cnn_version=CFG.input_model_cnn, fold=0, epoch=-1) # only last epoch saved
    print(list(model.conv1.parameters()))

In [ ]:
# Evaluate the custom model on a few validation samples

if CFG.evaluation:
    X = train.img_path.values
    y = train.target.values

    sample_size = 20
    train_index = 0 # some random image
    train_images = X[train_index:train_index+sample_size]
    train_targets = y[train_index:train_index+sample_size]
    train_dataset = ClassificationDataset(
        img_paths=train_images,
        targets=train_targets,
        original_indexes=None,
        tr=vtransform)

    FIG_SIZE = 6

    model.eval() # from model_make_custom above

    for i in range(sample_size):
        image_target = train_dataset[i]
        image, target = image_target['image'].unsqueeze(0), image_target['target']
        if target == torch.tensor(1.0):
            output = model(image).view(-1)
            print(output.detach().numpy(), target.detach().numpy(), image_target['img_id'])

            plt.figure(figsize=(FIG_SIZE, FIG_SIZE))
            plt.axes().yaxis.set_major_locator(ticker.MultipleLocator(40))
            plt.imshow(image.squeeze(0).numpy().transpose((1, 2, 0))[:,:,0]) # only 1 axis
            plt.title(f'target: {target}')
            plt.show()

# Utils

In [ ]:
def get_score(y_true, y_pred):
    score = roc_auc_score(y_true, y_pred)
    return score

# Training with folds

In [ ]:
def train_fn(data_loader, model, optimizer, criterion, device):
    
    model.train()
    
    for data in tqdm(data_loader, position=0, leave=True, desc='Training'):
        inputs = data['image']
        targets = data['target']
        
        inputs = inputs.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        targets = targets.unsqueeze(1)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()        

In [ ]:
def eval_fn(data_loader, model, device):
    
    model.eval()
    
    final_outputs = []
    final_targets = []
    final_img_ids = []
    
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            inputs = data['image']
            targets = data['target']
            img_ids = data['img_id']

            inputs = inputs.to(device, dtype=torch.float)
            output = model(inputs)
            
            output = output.detach().cpu().numpy().tolist()
            targets = targets.numpy().tolist()

            final_outputs.extend(output)
            final_targets.extend(targets)
            final_img_ids.extend(img_ids)
            
    return final_outputs, final_targets, final_img_ids

In [ ]:
# Train models for each fold

if CFG.training:
    models = []

    X = train.img_path.values
    y = train.target.values
    i = train.original_index.values

    skf = StratifiedKFold(n_splits=CFG.N_FOLDS)

    fold = 0
    for train_index, valid_index in skf.split(X, y):
        print(f"Starting Fold {fold:02d}")

        train_images, valid_images = X[train_index], X[valid_index]
        train_targets, valid_targets = y[train_index], y[valid_index]
        train_original_indexes = i[train_index]

        train_dataset = ClassificationDataset(
            img_paths=train_images,
            targets=train_targets,
            original_indexes=train_original_indexes,
            tr=ttransform
        )
        valid_dataset = ClassificationDataset(
            img_paths=valid_images,
            targets=valid_targets,
            original_indexes=None,
            tr=vtransform
        )

        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True , num_workers=CFG.num_workers)
        valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)

        model = model_make(CFG.model_name)
        model.to(device)

        # TODO higher lr for the conv1 parameters
        optimizer = torch.optim.Adam(model.parameters(), lr=CFG.lr)

        for epoch in range(CFG.epochs):
            train_fn(train_loader, model, optimizer, CFG.criterion, device=device)
            predictions, valid_targets, _ = eval_fn(valid_loader, model, device=device)
            roc_auc = get_score(valid_targets, predictions)
            print(f"Epoch={epoch}, Valid ROC AUC={roc_auc}")
            # print(list(model.conv1.parameters()))

            # Save model after each fold and epoch
            torch.save({'model': model.state_dict()},
                       OUTPUT_DIR+f"{CFG.model_name}_fold_{fold:02d}_epoch_{epoch:02d}_state.pth")

        # append the latest model
        # TODO: select the "best" model (after each epoch), not the last epoch
        models.append(model)
        fold += 1

        # for runs without submission, do only 1 fold
        if (CFG.submission == False) and (fold == 1):
            break

In [ ]:
# Prepare a submission on test data

if CFG.submission:    
    preds = []
    for model in models:
        test_dataset = ClassificationDataset(
            img_paths=test.img_path.values,
            targets=test.target.values,
            original_indexes=None,        
            tr=vtransform
        )
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=CFG.inference_batch_size, shuffle=False, num_workers=CFG.num_workers)
        predictions, valid_targets, _ = eval_fn(test_loader, model, device=device)
        preds.append(predictions)

In [ ]:
# average the inferences on the 4 fold models

if CFG.submission:
    outputs = []
    for predictions in preds:
        predictions = np.array(predictions)[:, 0]
        sig = torch.nn.Sigmoid()
        outs = sig(torch.from_numpy(predictions))
        outs = outs.detach().numpy()
        outputs.append(outs)

    # TODO: given the asymmetric nature of the signal (a small signal is still a signal); non-linear averaging might be more relevant here ???
    outputs = np.mean(outputs, axis = 0)
    display(outputs[0:10])

In [ ]:
if CFG.submission:
    test.target = outputs
    test.drop(['img_path'], axis=1, inplace=True)
    test.to_csv('submission.csv', index=False)

In [ ]:
# Evaluate models on validation AND training data for each fold and epoch
# try to see if and when overfitting occurs

if CFG.evaluation:
    # read back models from saved files in earlier runs
    import csv

    valid_results = []
    train_results = []

    X = train.img_path.values
    y = train.target.values
    skf = StratifiedKFold(n_splits=CFG.N_FOLDS)

    fold = 0
    for train_index, valid_index in skf.split(X, y):
        print(f"Starting Fold {fold:02d}")

        print(len(train_index), len(valid_index))
        print(train_index[0:5])
        print(valid_index[0:5])

        train_images, valid_images = X[train_index], X[valid_index]
        train_targets, valid_targets = y[train_index], y[valid_index]

        train_dataset = ClassificationDataset(
            img_paths=train_images,
            targets=train_targets,
            original_indexes=None,
            tr=vtransform)  # also vtransform since validation
        
        valid_dataset = ClassificationDataset(
            img_paths=valid_images,
            targets=valid_targets,
            original_indexes=None,
            tr=vtransform)

        # Here, since validation, shuffle False
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)
        valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)

        valid_results_per_fold = []
        train_results_per_fold = []

        # First 5 (0,1,2,3,4) from CNN-22
        # for epoch in range(CFG.epochs):
          
        # For CNN-18 only the data last fold "05" is saved
        for epoch in [5]:
            # DEBUG !!! 
            epoch_for_model = -1
            model = model_make_custom(CFG.model_name, CFG.input_model_cnn, fold=fold, epoch=epoch_for_model)
            model.to(device)

            # Validation over VALID data
            predictions, targets, img_ids = eval_fn(valid_loader, model, device=device)
            roc_auc = get_score(targets, predictions)
            print(f"Epoch={epoch}, Valid ROC AUC={roc_auc}")

            # predictions need to be flattened
            flat_predictions = []
            list(map(flat_predictions.extend, predictions))
            valid_results_per_fold_per_epoch = np.dstack((flat_predictions, targets, img_ids))[0]

            filename = OUTPUT_DIR+f"{CFG.model_name}_fold_{fold:02d}_epoch_{epoch:02d}_validation.csv"

            with open(filename, 'w') as f:
                csv.writer(f).writerows(valid_results_per_fold_per_epoch)

            valid_results_per_fold.append(valid_results_per_fold_per_epoch)

            # Validation over TRAIN data
            predictions, targets, img_ids = eval_fn(train_loader, model, device=device)
            roc_auc = get_score(targets, predictions)
            print(f"Epoch={epoch}, Train ROC AUC={roc_auc}")

            # predictions need to be flattened
            flat_predictions = []
            list(map(flat_predictions.extend, predictions))
            train_results_per_fold_per_epoch = np.dstack((flat_predictions, targets, img_ids))[0]

            filename = OUTPUT_DIR+f"{CFG.model_name}_fold_{fold:02d}_epoch_{epoch:02d}_training.csv"

            with open(filename, 'w') as f:
                csv.writer(f).writerows(train_results_per_fold_per_epoch)

            train_results_per_fold.append(train_results_per_fold_per_epoch)

        valid_results.append(valid_results_per_fold)
        train_results.append(train_results_per_fold)

        fold += 1

        # DEBUG only needed for 2 folds
        if fold == 2:
            break

In [ ]:
# Read back Inference of models on validation AND training data for each fold and epoch
# try to see if and when overfittigng occurs

if CFG.visualisation:
    
    INPUT_DIR = f"/kaggle/input/seti-signal-search-cnn-{CFG.input_inferences_cnn}/output"

    # DEBUG : Read back form previous cell, is small data
    INPUT_DIR = OUTPUT_DIR 
    
    import csv

    valid_results = []
    train_results = []

    X = train.img_path.values
    y = train.target.values
    skf = StratifiedKFold(n_splits=CFG.N_FOLDS) # we still hope this is reliable ...

    fold = 0
    for train_index, valid_index in skf.split(X, y):    
        print(f"Starting Fold {fold:02d}")

        print(len(train_index), len(valid_index))
        print(train_index[0:5])
        print(valid_index[0:5])

        train_images, valid_images = X[train_index], X[valid_index]
        train_targets, valid_targets = y[train_index], y[valid_index]

        valid_results_per_fold = []
        train_results_per_fold = []

        #for epoch in range(CFG.epochs):
        
        # DEBUG only fold 5 for now
        for epoch in [5]:
            
            # Validation over VALID data
            valid_results_per_fold_per_epoch = []
            filename = f"{INPUT_DIR}/{CFG.model_name}_fold_{fold:02d}_epoch_{epoch:02d}_validation.csv"
            with open(filename, 'r') as f:
                data_reader = csv.reader(f)
                for row in data_reader:
                    valid_results_per_fold_per_epoch.append(row)

            df_valid_results_per_fold_per_epoch = (
                pd.DataFrame(valid_results_per_fold_per_epoch, columns = ['predictions', 'targets', 'img_ids'])
            ) 
            valid_results_per_fold.append(valid_results_per_fold_per_epoch)

            roc_auc = get_score(df_valid_results_per_fold_per_epoch['targets'], df_valid_results_per_fold_per_epoch['predictions'])
            print(f"Epoch={epoch}, Valid ROC AUC={roc_auc}")

            # Validation over TRAIN data
            train_results_per_fold_per_epoch = []
            filename = f"{INPUT_DIR}/{CFG.model_name}_fold_{fold:02d}_epoch_{epoch:02d}_training.csv"
            with open(filename, 'r') as f:
                data_reader = csv.reader(f)
                for row in data_reader:
                    train_results_per_fold_per_epoch.append(row)

            df_train_results_per_fold_per_epoch = (
                pd.DataFrame(train_results_per_fold_per_epoch, columns = ['predictions', 'targets', 'img_ids'])
            ) 
            train_results_per_fold.append(train_results_per_fold_per_epoch)

            roc_auc = get_score(df_train_results_per_fold_per_epoch['targets'], df_train_results_per_fold_per_epoch['predictions'])
            print(f"Epoch={epoch}, Train ROC AUC={roc_auc}")

        valid_results.append(valid_results_per_fold)
        train_results.append(train_results_per_fold)

        fold += 1

        # DEBUG only 2 folds have this models calculated
        if fold == 2:
            break

In [ ]:
if CFG.visualisation:
    display(len(df_valid_results_per_fold_per_epoch))
    display(len(df_train_results_per_fold_per_epoch))

In [ ]:
if CFG.visualisation:
    fpr_valid, tpr_valid, threshold_valid = roc_curve(df_valid_results_per_fold_per_epoch['targets'].astype(float).astype(int), df_valid_results_per_fold_per_epoch['predictions'].astype(float))
    fpr_train, tpr_train, threshold_train = roc_curve(df_train_results_per_fold_per_epoch['targets'].astype(float).astype(int), df_train_results_per_fold_per_epoch['predictions'].astype(float))

    roc_auc_valid = get_score(df_valid_results_per_fold_per_epoch['targets'], df_valid_results_per_fold_per_epoch['predictions'])
    roc_auc_train = get_score(df_train_results_per_fold_per_epoch['targets'], df_train_results_per_fold_per_epoch['predictions'])

    print(f"roc_auc_valid = {roc_auc_valid}")
    print(f"roc_auc_train = {roc_auc_train}")
    FIG_SIZE = 8

    plt.subplots(1, figsize=(FIG_SIZE, FIG_SIZE))

    plt.title('ROC - validation data')
    plt.plot(fpr_valid, tpr_valid, 'b')
    #plt.plot(fpr_valid, threshold_valid / 10.0, 'r')
    plt.plot([0, 1], ls="--")
    plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
    plt.xlabel("False Positive Rate" , fontsize=12)
    plt.ylabel("True Positive Rate" , fontsize=12)

    plt.subplots(1, figsize=(FIG_SIZE, FIG_SIZE))

    plt.title('ROC - train data')
    plt.plot(fpr_train, tpr_train, 'b')
    #plt.plot(fpr_train, threshold_train / 10.0, 'r')
    plt.plot([0, 1], ls="--")
    plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
    plt.xlabel("False Positive Rate" , fontsize=12)
    plt.ylabel("True Positive Rate" , fontsize=12)


    plt.show()